In [ ]:
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_source_control_pane()

Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 7.36 KiB | 7.36 MiB/s, done.
Resolving deltas: 100% (7/7), done.


interactive(children=(Text(value='', description='Email', placeholder='Enter email'), Text(value='', descripti…

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import importlib
import eeddf
import bqddf
importlib.reload(eeddf)
importlib.reload(bqddf)

eeddf.initialize_ddf(test_environment = True)

In [10]:
from google.cloud import bigquery
import time

In [11]:
client = bqddf._get_big_query_client()

In [12]:
# Create a minimal metadata schema
metadata_schema = [
    bigquery.SchemaField("training_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("num_max_epochs", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("num_layers", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("num_nodes_per_layer", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("training_batch_size", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("learning_rate", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("early_stopping_patience", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("kl_num_samples_from_pred_dist", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("dropout_rate", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("resolution_x", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("resolution_y", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("mean_label", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("var_label", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("double_sided_kl", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("activation_func", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("features_to_standardize", "STRING", mode="REPEATED"),
    bigquery.SchemaField("features_to_passthrough", "STRING", mode="REPEATED"),
    bigquery.SchemaField("raw_metadata", "JSON", mode="NULLABLE")
]

In [14]:
# Modify config values for test.
bqddf._CONFIG['TRAINING_METADATA_TABLE'] = f'test_{int(time.time())}'

In [20]:
# table_id should evaluate to project.dataset.test-table
table_id = f"{bqddf._CONFIG['PROJECT_NAME']}.{bqddf._CONFIG['DATASET']}.{bqddf._CONFIG['TRAINING_METADATA_TABLE']}"
table = bigquery.Table(table_id, schema=metadata_schema)

In [21]:
# If it exists, delete it first.
client.delete_table(table_id, not_found_ok=True)

In [41]:
# Create the test table.
table = client.create_table(table)
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table river-sky-386919.harness_test_db.training_runs


In [26]:
import json

In [38]:
def test_read_and_write():
  training_metadata = {
    "num_max_epochs" : 20,
    "mean_label" : "mean_label_1_fake"
  }

  expected_training_id = bqddf._generate_training_id(training_metadata)
  assert bqddf.insert_training_metadata(training_metadata) == expected_training_id

  result = bqddf.get_training_result(expected_training_id)
  assert result.total_rows == 1

In [39]:
test_read_and_write()

In [40]:
client.delete_table(table_id)